In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=ChatGroq(model="llama-3.1-8b-instant")

result=llm.invoke("HEllo!")
result


AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 39, 'total_tokens': 49, 'completion_time': 0.010987368, 'prompt_time': 0.001888136, 'queue_time': 0.047248664, 'total_time': 0.012875504}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_3ddc9808b3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c43eaf66-f907-4edb-a0a1-9b5d6e013e71-0', usage_metadata={'input_tokens': 39, 'output_tokens': 10, 'total_tokens': 49})

In [10]:
from typing import Annotated
import operator
from typing_extensions import Literal
from pydantic import BaseModel,Field
from langchain_core.messages import HumanMessage,SystemMessage
from typing_extensions import TypedDict

class State(TypedDict):
    joke:str
    topic:str
    feedback:str
    funny_or_not:str





In [11]:
class feedback(BaseModel):
    garde:Literal["funny","not funny"]=Field(
        description="Decide if the joke is funny or not funnny"
    )
    
    feedback:str=Field(
        description="Tf the joke is not funny, provide feedback to improve it.",
    )

evaluator=llm.with_structured_output(feedback)

In [12]:
def llm_call_generator(state:State):
    """LLM generate a joke"""
    # LLM generates a joke
    if State.get("feedback"):
        msg = llm.invoke(
            f"Write a joke about {state['topic']} but take into account the feedback: {state['feedback']}"
    )
    else:
        msg = llm.invoke(f"Write a joke about {state['topic']}")
    return {"joke": msg.content}


def llm_call_evaluator(state: State):
    """LLM evaluates the joke"""
    grade = evaluator.invoke(f"Grade the joke {state['joke']}")
    return {"funny_or_not": grade.grade, "feedback": grade.feedback}


In [13]:
def route_joke(state:State):
    """Route back to joke generator or end based upon feedback from the evaluator"""

    if state["funny_or_not"]=="funny":
        return "Accepted"
    elif state["funny_or_not"]=="not_funny":
        return "Rejected + Feedback"
    

In [16]:
from langgraph.graph import StateGraph,START,END
from IPython.display import display,Image

optimizer_buillder= StateGraph(State)

optimizer_buillder.add_node("llm_call_generator",llm_call_generator)
optimizer_buillder.add_node("llm_call_evaluator",llm_call_evaluator)

optimizer_buillder.add_edge(START,"llm_call_generator")
optimizer_buillder.add_edge("llm_call_generator","llm_call_evaluator")
optimizer_buillder.add_conditional_edges(
    "llm_call_evaluator",
    route_joke,
    {
        "Accepted":END,
        "Rejected + Feedback ":"llm_call_generator",
    },
)

optimizer_workflow=optimizer_buillder.compile()

display(Image(optimizer_workflow.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`